In [1]:
from __future__ import unicode_literals
from textacy import fileio, preprocess, extract, Corpus

In [2]:
summary_docs = fileio.read_file_lines('/home/SMRT-labeled/summary-docs')
summary_meta = fileio.read_json_lines('/home/SMRT-labeled/summary-meta-relabel')

In [3]:
text = [t for t in summary_docs]
meta = [m for m in summary_meta][0]

corpus_train = Corpus('en', texts = text, metadatas = meta)

In [4]:
for k in meta[0]:
    print k, meta[0][k]

Category smrt corporate
Topic corporate
Tonality neutral


In [5]:
#topic model
from textacy import vsm

terms_lists = (doc.to_terms_list(ngrams={2, 3}, named_entities=True, as_strings=True) for doc in corpus_train)
doc_term_matrix, id2term = vsm.doc_term_matrix(terms_lists, weighting='tfidf', normalize=True, smooth_idf=True, min_df=2, max_df=0.80, max_n_terms=15000)

In [6]:
#load lable data
label_key = u'Topic'

label_list = map(lambda x:x[label_key], meta)
unique_label = list(set(label_list))
label2id = dict(zip(unique_label, range(len(unique_label))))
label_id = map(lambda x:label2id[x], label_list)

In [7]:
doc_term_matrix

<1766x10607 sparse matrix of type '<type 'numpy.float64'>'
	with 46405 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.svm import SVC
import numpy as np

#SVM classifier
msg_x = doc_term_matrix.toarray()
msg_y = np.array(label_id)

msg_model = SVC(kernel=str('linear'))
msg_model.fit(msg_x, msg_y)

#evaluate the accuracy in training set
print("Accuracy in training set:"+ str(msg_model.score(msg_x, msg_y)))

Accuracy in training set:0.931483578709


In [9]:
#save model
from sklearn.externals import joblib

savefiles = joblib.dump(msg_model, '/home/pretrain-model/topic.pkl') 

#save meta
fileio.write_json(label2id, '/home/pretrain-model/topic-label2id')
fileio.write_json(id2term, '/home/pretrain-model/topic-id2term')